In [1]:
#importing dataset
import pandas as pd
df= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\train.csv")
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
#preprocessing 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import string

df['new_tweet'] = df.tweet.str.replace('@user', '')
df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )
df['new_tweet'] = df['new_tweet'].str.replace("#", " ")
df['new_tweet'] = df['new_tweet'].apply(lambda x: x.split())
df['new_tweet']= df['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords]
df['new_tweet'] = df['new_tweet'].apply(process)
df=df.drop(columns='tweet')
df=df.drop(columns='id')
df.head()


<ipython-input-2-d574088bc6ff>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )


,label,new_tweet
0,0,"[dysfunct, drag, selfish, father, kid, run]"
1,0,"[disapoint, lyft, caus, getthank, van, use, pd..."
2,0,"[bihday, majesti]"
3,0,"[model, time, love, ur, take, u]"
4,0,"[factsguid, motiv, societi]"


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test =train_test_split(df["new_tweet"], df["label"], test_size = 0.2, random_state = 42)


In [4]:
import numpy as np
split=np.array_split(x_train, 10)

In [5]:
label=np.array_split(y_train,10)

In [6]:
from scipy.stats import entropy
from typing import Union, List, Sequence
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.base import BaseEstimator
import numpy as np
def multi_argmax(values: np.ndarray, n_instances: int = 50) -> np.ndarray:
    assert n_instances <= values.shape[0], 'n_instances must be less or equal than the size of utility'
    max_idx = np.argpartition(-values, n_instances-1, axis=0)[:n_instances]
    return max_idx

def shuffled_argmax(values: np.ndarray, n_instances: int = 50) -> np.ndarray:

    assert n_instances <= values.shape[0], 'n_instances must be less or equal than the size of utility'

    # shuffling indices and corresponding values
    shuffled_idx = np.random.permutation(len(values))
    shuffled_values = values[shuffled_idx]

    # getting the n_instances best instance
    # since mergesort is used, the shuffled order is preserved
    sorted_query_idx = np.argsort(shuffled_values, kind='mergesort')[len(shuffled_values)-n_instances:]

    # inverting the shuffle
    query_idx = shuffled_idx[sorted_query_idx]
    return query_idx


modALinput = Union[sp.csr_matrix, pd.DataFrame, np.ndarray, list]


def data_vstack(blocks: Sequence[modALinput]) -> modALinput:
    if any([sp.issparse(b) for b in blocks]):
        return sp.vstack(blocks)
    elif isinstance(blocks[0], pd.DataFrame):
        return blocks[0].append(blocks[1:])
    elif isinstance(blocks[0], np.ndarray):
        return np.concatenate(blocks)
    elif isinstance(blocks[0], list):
        return np.concatenate(blocks).tolist()

    raise TypeError('%s datatype is not supported' % type(blocks[0]))


def retrieve_rows(X: modALinput,I: Union[int, List[int], np.ndarray]) -> Union[sp.csc_matrix, np.ndarray, pd.DataFrame]:
    if sp.issparse(X):
        try:
            return X[I]
        except:
            sp_format = X.getformat()
            return X.tocsr()[I].asformat(sp_format)
    elif isinstance(X, pd.DataFrame):
        return X.iloc[I]
    elif isinstance(X, np.ndarray):
        return X[I]
    elif isinstance(X, list):
        return np.array(X)[I].tolist()

    raise TypeError('%s datatype is not supported' % type(X))



In [7]:

def classifier_entropy(classifier: BaseEstimator, X: modALinput, **predict_proba_kwargs) -> np.ndarray:
    try:
        classwise_uncertainty = classifier.predict_proba(X, **predict_proba_kwargs)
    except NotFittedError:
        return np.zeros(shape=(X.shape[0], ))

    return np.transpose(entropy(np.transpose(classwise_uncertainty)))

def entropy_sampling(classifier: BaseEstimator, X: modALinput,
                     n_instances: int = 50, random_tie_break: bool = False,
                     **uncertainty_measure_kwargs) -> np.ndarray:
    entropy = classifier_entropy(classifier, X, **uncertainty_measure_kwargs)
    if not random_tie_break:
        return multi_argmax(entropy, n_instances=n_instances)
        
    return shuffled_argmax(entropy, n_instances=n_instances)

    

In [8]:
import abc
import sys
import warnings
from typing import Union, Callable, Optional, Tuple, List, Iterator, Any

import numpy as np
from sklearn.base import BaseEstimator
from sklearn.utils import check_X_y

import scipy.sparse as sp


if sys.version_info >= (3, 4):
    ABC = abc.ABC
else:
    ABC = abc.ABCMeta('ABC', (), {})


class ActiveLearner(ABC, BaseEstimator):

    def __init__(self,
                 estimator: BaseEstimator,
                 query_strategy: Callable,
                 X_training: Optional[modALinput] = None,
                 y_training: Optional[modALinput] = None,
                 bootstrap_init: bool = False,
                 on_transformed: bool = False,
                 force_all_finite: bool = True,
                 **fit_kwargs
                 ) -> None:
        assert callable(query_strategy), 'query_strategy must be callable'

        self.estimator = estimator
        self.query_strategy = query_strategy
        self.on_transformed = on_transformed

        self.X_training = X_training
        self.y_training = y_training
        if X_training is not None:
            self._fit_to_known(bootstrap=bootstrap_init, **fit_kwargs)

        assert isinstance(force_all_finite, bool), 'force_all_finite must be a bool'
        self.force_all_finite = force_all_finite

    def _add_training_data(self, X: modALinput, y: modALinput) -> None:
        check_X_y(X, y, accept_sparse=True, ensure_2d=False, allow_nd=True, multi_output=True, dtype=None,
                  force_all_finite=self.force_all_finite)

        if self.X_training is None:
            self.X_training = X
            self.y_training = y
        else:
            try:
                self.X_training = data_vstack((self.X_training, X))
                self.y_training = data_vstack((self.y_training, y))
            except ValueError:
                raise ValueError('the dimensions of the new training data and label must'
                                 'agree with the training data and labels provided so far')

    def _fit_to_known(self, bootstrap: bool = False, **fit_kwargs) -> 'BaseLearner':
        if not bootstrap:
            self.estimator.fit(self.X_training, self.y_training, **fit_kwargs)
        else:
            n_instances = self.X_training.shape[0]
            bootstrap_idx = np.random.choice(range(n_instances), n_instances, replace=True)
            self.estimator.fit(self.X_training[bootstrap_idx], self.y_training[bootstrap_idx], **fit_kwargs)

        return self

    def predict(self, X: modALinput, **predict_kwargs) -> Any:
        return self.estimator.predict(X, **predict_kwargs)

    def predict_proba(self, X: modALinput, **predict_proba_kwargs) -> Any:
        return self.estimator.predict_proba(X, **predict_proba_kwargs)

    def query(self, X_pool, *query_args, **query_kwargs) -> Union[Tuple, modALinput]:
        query_result = self.query_strategy(self, X_pool, *query_args, **query_kwargs)

        return query_result, retrieve_rows(X_pool, query_result)
    
    def teach(self, X: modALinput, y: modALinput, bootstrap: bool = False, only_new: bool = False, **fit_kwargs) -> None:

        self._add_training_data(X, y)
        if not only_new:
            self._fit_to_known(bootstrap=bootstrap, **fit_kwargs)
        else:
            self._fit_on_new(X, y, bootstrap=bootstrap, **fit_kwargs)

In [9]:
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(split[0].apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [10]:

learner = ActiveLearner(estimator=RandomForestClassifier(),query_strategy=entropy_sampling,X_training=x_train_tfidf, y_training=label[0].ravel())
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions = learner.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

94.10292507430002


In [11]:
for i in range(1,10):
    x_counts = count_vect.transform(split[i].apply(lambda x: ' '.join(x)))
    x_tfidf = transformer.transform(x_counts)
    query_idx, query_sample = learner.query(x_tfidf)
      #print(query_idx)
      #print(query_sample)
    split[i]=split[i].reset_index()
    split[i]=split[i].copy()
    split[i]=split[i].drop(['index'],axis=1)
    y=[]
      #print(split[i].loc[query_idx])
    for k in query_idx:
      j=label[i].iloc[k]
      #print(x_tfidf[query_idx])
      lb=j.item()
      y.append(lb)
      #inp=int(input("Enter the label"))
    learner.teach(x_tfidf[query_idx],y)
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions = learner.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)
      

95.19787267323635
